In [87]:
# Import packages
import pandas as pd
import numpy as np

# Teammate 1 (Preprocessing)

In [ ]:
# Import packages for BigQuery access
!pip install --upgrade google-cloud-bigquery
!pip install google-cloud-bigquery[bqstorage,pandas]
from google.cloud import bigquery
from google.colab import auth
from google.colab import files

In [89]:
# Initialize BigQuery client connected to the MIMIC-IV-EXT-FAIRGRAD-EVAL project
auth.authenticate_user()
client = bigquery.Client(project="mimic-iv-ext-fairgrad-eval")

In [90]:
def q(query_string):
    query_job = client.query(query_string)
    return query_job.to_dataframe()

### Recreating the researcher's sepsis cohort: 
Criteria:
- Must have sepsis
- Must be admitted to ED or ICU at Beth Israel Deaconess Medical Center
- No NULL values
- Race must be known

Features to join:
- In-hospital mortality (deceased vs. survived)
- Age
- Heart Rate
- Glucose
- Systolic Blood Pressure(SBP)
- Charlson Comorbidity Index (CCI)
- APACHE-III score
- Race
- Sex

In [91]:
# In-hospital mortality
# Gender
# Age
# Race

q("""SELECT subject_id, hadm_id, stay_id, gender, admission_age, race, hospital_expire_flag, first_hosp_stay, first_icu_stay
FROM `physionet-data.mimiciv_3_1_derived.icustay_detail` 
WHERE subject_id IS NOT NULL
AND hadm_id IS NOT NULL
AND stay_id IS NOT NULL
AND gender IS NOT NULL
AND admission_age IS NOT NULL
AND race IS NOT NULL
AND hospital_expire_flag IS NOT NULL
LIMIT 10""")

,subject_id,hadm_id,stay_id,gender,admission_age,race,hospital_expire_flag,first_hosp_stay,first_icu_stay
0,10141487,24889188,39804513,F,18,UNKNOWN,0,True,True
1,10168562,20936196,30883841,F,18,WHITE,0,True,True
2,10307127,21399795,36366751,M,18,BLACK/AFRICAN AMERICAN,0,True,False
3,10307127,21399795,35486091,M,18,BLACK/AFRICAN AMERICAN,0,True,True
4,10391698,25190401,39250200,F,18,WHITE,0,True,True
5,10542217,22402134,37079602,M,18,WHITE,0,True,True
6,10561852,25577419,34454262,F,18,UNABLE TO OBTAIN,0,True,True
7,10562205,20149612,34317198,M,18,WHITE,0,True,True
8,10652259,24797020,33359917,M,18,UNKNOWN,0,True,True
9,10665087,25197235,30970959,F,18,UNKNOWN,0,True,True


In [92]:
# Heart Rate
# Glucose
# Systolic Blood Pressure (SBP)

q("""
SELECT subject_id, stay_id, heart_rate_min, heart_rate_max, 
heart_rate_mean, glucose_min, glucose_max, glucose_mean, sbp_min, 
sbp_max, sbp_mean 
FROM `physionet-data.mimiciv_3_1_derived.first_day_vitalsign` 
WHERE subject_id IS NOT NULL
and stay_id IS NOT NULL
and heart_rate_min IS NOT NULL
and heart_rate_max IS NOT NULL
and heart_rate_mean IS NOT NULL
and sbp_min IS NOT NULL
and sbp_max IS NOT NULL
and sbp_mean IS NOT NULL
and heart_rate_min IS NOT NULL
and heart_rate_max IS NOT NULL
and heart_rate_mean IS NOT NULL
and glucose_min IS NOT NULL
and glucose_max IS NOT NULL
and glucose_mean IS NOT NULL
LIMIT 10
""")

,subject_id,stay_id,heart_rate_min,heart_rate_max,heart_rate_mean,glucose_min,glucose_max,glucose_mean,sbp_min,sbp_max,sbp_mean
0,14106509,35818526,57.0,141.0,102.625000,64.0,73.0,68.5,44.0,97.0,80.600000
1,14356315,37003247,101.0,123.0,110.571429,81.0,81.0,81.0,75.0,125.0,100.000000
2,19349187,37874808,30.0,132.0,104.666667,83.0,83.0,83.0,76.0,113.0,92.333333
3,17513748,37283456,56.0,69.0,64.357143,84.0,84.0,84.0,101.0,156.0,117.666667
4,13630653,38685798,110.0,110.0,110.000000,79.0,85.0,82.0,195.0,195.0,195.000000
5,19948220,39201286,58.0,145.0,108.333333,97.0,97.0,97.0,72.0,201.0,134.400000
6,18102901,30488976,70.0,101.0,86.058824,113.0,113.0,113.0,104.0,132.0,117.444444
7,18265680,39346605,87.0,107.0,95.700000,116.0,116.0,116.0,109.0,145.0,120.000000
8,13788454,36414583,94.0,133.0,114.666667,122.0,122.0,122.0,59.0,108.0,81.000000
9,16324127,33732908,95.0,119.0,108.400000,127.0,127.0,127.0,30.0,118.0,85.000000


In [93]:
# Charlson Comorbidity Index (CCI)
q("""
SELECT subject_id, hadm_id, charlson_comorbidity_index 
FROM `physionet-data.mimiciv_3_1_derived.charlson` 
WHERE subject_id IS NOT NULL
AND hadm_id IS NOT NULL
AND charlson_comorbidity_index IS NOT NULL
LIMIT 10
""")

,subject_id,hadm_id,charlson_comorbidity_index
0,13521437,21043620,5
1,15754851,24589551,4
2,11869791,23452974,0
3,18828736,25260719,6
4,19300198,20183219,4
5,18151168,20556551,8
6,19585183,21531681,5
7,15862493,24138206,7
8,16231014,27637020,3
9,15767642,26994406,6


In [94]:
# APACHE-III score
q("""
SELECT subject_id, hadm_id, stay_id, apsiii
FROM `physionet-data.mimiciv_3_1_derived.apsiii` 
WHERE subject_id IS NOT NULL
AND hadm_id IS NOT NULL
AND stay_id IS NOT NULL
AND apsiii IS NOT NULL
LIMIT 10
""")


,subject_id,hadm_id,stay_id,apsiii
0,18944959,22112997,32694803,0
1,14766306,21640292,32321012,0
2,17676327,27456005,37960207,0
3,17784939,28330780,32000668,0
4,10971284,22087634,33084533,0
5,11618742,27083914,38146761,0
6,11033454,20410256,38852341,0
7,18857002,28090043,32379958,0
8,17861096,28033870,34455374,0
9,18016793,22565957,35019933,0


In [95]:
# Sepsis diagnosis criteria

q("""
SELECT
  di.subject_id, di.hadm_id,
  STRING_AGG(DISTINCT d.long_title, '; ' ORDER BY d.long_title) AS sepsis_dx_titles
FROM `physionet-data.mimiciv_3_1_hosp.diagnoses_icd` AS di
JOIN `physionet-data.mimiciv_3_1_hosp.d_icd_diagnoses` AS d
  ON di.icd_code = d.icd_code
 AND di.icd_version = d.icd_version
WHERE LOWER(d.long_title) LIKE '%sepsis%'
AND di.subject_id IS NOT NULL
AND di.hadm_id IS NOT NULL
GROUP BY di.subject_id, di.hadm_id
ORDER BY di.subject_id, di.hadm_id
LIMIT 10
""")

,subject_id,hadm_id,sepsis_dx_titles
0,10000826,21086876,Sepsis
1,10001401,27012892,Sepsis due to Enterococcus; Sepsis following a...
2,10001401,27060146,"Sepsis, unspecified organism"
3,10001843,26133978,"Sepsis, unspecified organism; Severe sepsis wi..."
4,10002013,24848509,Severe sepsis
5,10002013,25442395,"Sepsis, unspecified organism"
6,10002428,23473524,Sepsis
7,10002428,28662225,Severe sepsis
8,10003019,22774359,Severe sepsis
9,10003400,23559586,"Sepsis, unspecified organism; Severe sepsis wi..."


### Join all relevant tables

Tables
- `physionet-data.mimiciv_3_1_hosp.diagnoses_icd`
- `physionet-data.mimiciv_3_1_hosp.d_icd_diagnoses`
- `physionet-data.mimiciv_3_1_derived.icustay_detail`
- `physionet-data.mimiciv_3_1_derived.first_day_vitalsign`
- `physionet-data.mimiciv_3_1_derived.charlson`
- `physionet-data.mimiciv_3_1_derived.apsiii`

Join On
- subject_id = patient (can have multiple hadm_id)
- hadm_id = hospital admission (can have multiple stay_id)
- stay_id = icu admission

In [96]:
# Join all tables above on subject_id, hadm_id, and stay_id to create full feature dataset

q("""SELECT 
  d.hospital_expire_flag,d.subject_id, d.hadm_id, d.stay_id, d.gender, d.admission_age, d.race, d.first_hosp_stay, d.first_icu_stay,
  v.heart_rate_min, v.heart_rate_max, v.heart_rate_mean,
  v.glucose_min, v.glucose_max, v.glucose_mean,
  v.sbp_min, v.sbp_max, v.sbp_mean,
  c.charlson_comorbidity_index,
  a.apsiii
FROM `physionet-data.mimiciv_3_1_derived.icustay_detail` AS d
JOIN `physionet-data.mimiciv_3_1_derived.first_day_vitalsign` AS v
  ON d.subject_id = v.subject_id AND d.stay_id = v.stay_id
JOIN `physionet-data.mimiciv_3_1_derived.charlson` AS c
  ON d.subject_id = c.subject_id AND d.hadm_id = c.hadm_id
JOIN `physionet-data.mimiciv_3_1_derived.apsiii` AS a
  ON d.subject_id = a.subject_id AND d.hadm_id = a.hadm_id AND d.stay_id = a.stay_id
WHERE d.subject_id IS NOT NULL
AND d.hadm_id IS NOT NULL
AND d.stay_id IS NOT NULL
AND d.gender IS NOT NULL
AND d.admission_age IS NOT NULL
AND d.race IS NOT NULL
AND d.hospital_expire_flag IS NOT NULL
AND d.first_hosp_stay IS NOT NULL
AND d.first_icu_stay IS NOT NULL
AND v.heart_rate_min IS NOT NULL
AND v.heart_rate_max IS NOT NULL
AND v.heart_rate_mean IS NOT NULL
AND v.glucose_min IS NOT NULL
AND v.glucose_max IS NOT NULL
AND v.glucose_mean IS NOT NULL
AND v.sbp_min IS NOT NULL
AND v.sbp_max IS NOT NULL
AND v.sbp_mean IS NOT NULL
AND c.charlson_comorbidity_index IS NOT NULL
AND a.apsiii IS NOT NULL
LIMIT 10
""")

,hospital_expire_flag,subject_id,hadm_id,stay_id,gender,admission_age,race,first_hosp_stay,first_icu_stay,heart_rate_min,heart_rate_max,heart_rate_mean,glucose_min,glucose_max,glucose_mean,sbp_min,sbp_max,sbp_mean,charlson_comorbidity_index,apsiii
0,0,10168562,20936196,30883841,F,18,WHITE,True,True,46.0,72.0,55.789474,97.0,101.0,99.000000,103.0,147.0,124.647059,1,20
1,0,10307127,21399795,36366751,M,18,BLACK/AFRICAN AMERICAN,True,False,103.0,142.0,120.192308,105.0,119.0,112.000000,111.0,138.0,122.833333,2,33
2,0,10307127,21399795,35486091,M,18,BLACK/AFRICAN AMERICAN,True,True,102.0,135.0,112.916667,110.0,113.0,111.500000,124.0,166.0,144.347826,2,22
3,0,10391698,25190401,39250200,F,18,WHITE,True,True,100.0,126.0,117.333333,97.0,289.0,191.913043,95.0,128.0,107.300000,1,43
4,0,10542217,22402134,37079602,M,18,WHITE,True,True,47.0,90.0,71.560000,79.0,79.0,79.000000,87.0,145.0,123.240000,0,27
5,0,10561852,25577419,34454262,F,18,UNABLE TO OBTAIN,True,True,67.0,107.0,82.370370,72.0,160.0,88.533333,73.0,118.0,102.730769,1,26
6,0,10562205,20149612,34317198,M,18,WHITE,True,True,97.0,132.0,116.800000,94.0,125.0,109.500000,74.0,131.0,104.653846,0,52
7,0,10652259,24797020,33359917,M,18,UNKNOWN,True,True,97.0,153.0,115.750000,80.0,123.0,103.666667,96.0,126.0,110.958333,1,83
8,0,10665087,25197235,30970959,F,18,UNKNOWN,True,True,54.0,89.0,68.842105,83.0,83.0,83.000000,107.0,152.0,122.052632,1,14
9,0,10731984,25707431,38983602,F,18,ASIAN - CHINESE,True,True,63.0,113.0,82.609375,99.0,174.0,135.333333,77.0,150.0,115.245614,1,36


In [97]:
# Join the sepsis diagnosis criteria to the full feature dataset to create a final dataset for modeling
# This will be the dataset we use for the rest of the project

df=q("""WITH sepsis_patients AS (
  SELECT
    di.subject_id, di.hadm_id,
    STRING_AGG(DISTINCT d.long_title, '; ' ORDER BY d.long_title) AS sepsis_dx_titles
  FROM `physionet-data.mimiciv_3_1_hosp.diagnoses_icd` AS di
  JOIN `physionet-data.mimiciv_3_1_hosp.d_icd_diagnoses` AS d
    ON di.icd_code = d.icd_code
   AND di.icd_version = d.icd_version
  WHERE LOWER(d.long_title) LIKE '%sepsis%'
  AND di.subject_id IS NOT NULL
  AND di.hadm_id IS NOT NULL
  GROUP BY di.subject_id, di.hadm_id
)

SELECT 
  d.hospital_expire_flag, d.subject_id, d.hadm_id, d.stay_id, d.gender, d.admission_age, d.race, d.first_hosp_stay, d.first_icu_stay,
  v.heart_rate_min, v.heart_rate_max, v.heart_rate_mean,
  v.glucose_min, v.glucose_max, v.glucose_mean,
  v.sbp_min, v.sbp_max, v.sbp_mean,
  c.charlson_comorbidity_index,
  a.apsiii,
  s.sepsis_dx_titles
FROM `physionet-data.mimiciv_3_1_derived.icustay_detail` AS d
JOIN `physionet-data.mimiciv_3_1_derived.first_day_vitalsign` AS v
  ON d.subject_id = v.subject_id AND d.stay_id = v.stay_id
JOIN `physionet-data.mimiciv_3_1_derived.charlson` AS c
  ON d.subject_id = c.subject_id AND d.hadm_id = c.hadm_id
JOIN `physionet-data.mimiciv_3_1_derived.apsiii` AS a
  ON d.subject_id = a.subject_id AND d.hadm_id = a.hadm_id AND d.stay_id = a.stay_id
JOIN sepsis_patients AS s
  ON d.subject_id = s.subject_id AND d.hadm_id = s.hadm_id
WHERE d.subject_id IS NOT NULL
AND d.hadm_id IS NOT NULL
AND d.stay_id IS NOT NULL
AND d.gender IS NOT NULL
AND d.admission_age IS NOT NULL
AND d.race IS NOT NULL
AND d.hospital_expire_flag IS NOT NULL
AND d.first_hosp_stay IS NOT NULL
AND d.first_icu_stay IS NOT NULL
AND v.heart_rate_min IS NOT NULL
AND v.heart_rate_max IS NOT NULL
AND v.heart_rate_mean IS NOT NULL
AND v.glucose_min IS NOT NULL
AND v.glucose_max IS NOT NULL
AND v.glucose_mean IS NOT NULL
AND v.sbp_min IS NOT NULL
AND v.sbp_max IS NOT NULL
AND v.sbp_mean IS NOT NULL
AND c.charlson_comorbidity_index IS NOT NULL
AND a.apsiii IS NOT NULL""")

df

,hospital_expire_flag,subject_id,hadm_id,stay_id,gender,admission_age,race,first_hosp_stay,first_icu_stay,heart_rate_min,...,heart_rate_mean,glucose_min,glucose_max,glucose_mean,sbp_min,sbp_max,sbp_mean,charlson_comorbidity_index,apsiii,sepsis_dx_titles
0,0,10562205,20149612,34317198,M,18,WHITE,True,True,97.0,...,116.800000,94.0,125.0,109.500000,74.0,131.0,104.653846,0,52,Severe sepsis
1,0,10731984,25707431,38983602,F,18,ASIAN - CHINESE,True,True,63.0,...,82.609375,99.0,174.0,135.333333,77.0,150.0,115.245614,1,36,Severe sepsis
2,0,13247025,23936736,31435087,F,18,WHITE,True,True,91.0,...,104.800000,86.0,89.0,87.500000,90.0,116.0,103.240000,0,32,Severe sepsis
3,0,16671253,27961922,33708525,F,18,HISPANIC/LATINO - MEXICAN,True,True,95.0,...,111.380952,85.0,85.0,85.000000,109.0,136.0,121.000000,0,42,Severe sepsis without septic shock
4,0,18134371,28733732,32559951,F,18,BLACK/CAPE VERDEAN,True,True,64.0,...,89.000000,92.0,113.0,103.333333,80.0,130.0,107.871795,0,40,Sepsis due to Escherichia coli [E. coli]; Seve...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16716,1,18208434,25509166,35137839,F,99,WHITE,False,True,41.0,...,96.365854,73.0,310.0,142.857143,75.0,146.0,111.300000,10,83,"Sepsis, unspecified organism; Severe sepsis wi..."
16717,1,12773529,28154720,39113872,F,100,WHITE,True,True,59.0,...,70.625000,168.0,168.0,168.000000,82.0,156.0,120.375000,7,42,"Sepsis, unspecified organism; Severe sepsis wi..."
16718,0,10086022,24567350,34048359,F,102,WHITE,True,True,81.0,...,101.512821,104.0,154.0,122.666667,83.0,180.0,112.157895,4,47,Severe sepsis with septic shock
16719,0,16853729,20799576,30779061,F,103,OTHER,False,True,57.0,...,76.642857,137.0,152.0,144.666667,63.0,126.0,99.717949,6,105,"Sepsis, unspecified organism; Severe sepsis wi..."


### Pre-Processing/ Cleaning

In [220]:
# Every row in the dataset corresponds to a unique subject_id, shuffle the dataset and drop duplicates to ensure this is the case

df_unique = (df
             .sample(frac=1, random_state=42)  # random shuffle
             .drop_duplicates(subset='subject_id', keep='first')
             .reset_index(drop=True))

print(f"Rows removed: {len(df)-len(df_unique)}")
print(f"Rows remaining:  {len(df_unique)}")
print(f"Unique subject_ids: {df_unique['subject_id'].nunique()}")

Rows removed: 4765
Rows remaining:  11956
Unique subject_ids: 11956


In [221]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11956 entries, 0 to 11955
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   hospital_expire_flag        11956 non-null  Int64  
 1   subject_id                  11956 non-null  Int64  
 2   hadm_id                     11956 non-null  Int64  
 3   stay_id                     11956 non-null  Int64  
 4   gender                      11956 non-null  object 
 5   admission_age               11956 non-null  Int64  
 6   race                        11956 non-null  object 
 7   first_hosp_stay             11956 non-null  boolean
 8   first_icu_stay              11956 non-null  boolean
 9   heart_rate_min              11956 non-null  float64
 10  heart_rate_max              11956 non-null  float64
 11  heart_rate_mean             11956 non-null  float64
 12  glucose_min                 11956 non-null  float64
 13  glucose_max                 119

#### Cleaning tasks

Drop columns:
- sepsis_dx_titles
- first_hosp_stay
- first_icu_stay

Rename columns:
- hospital_expire_flag -> target
- admission_age -> age
- charlson_comorbidity_index -> cci

Change types:
- hospital_expire_flag: Int64 -> boolean
- gender, race, sepsis_dx_titles: object -> string

In [231]:
df_clean = df_unique.copy()

# Drop columns
df_clean.drop(columns=['subject_id','hadm_id', 'stay_id','sepsis_dx_titles', 'first_hosp_stay', 'first_icu_stay'], inplace=True)

# Rename columns
df_clean.rename(columns={
    'hospital_expire_flag': 'target',
    'admission_age': 'age',
    'charlson_comorbidity_index': 'cci'}, inplace=True)

# Encode gender: Male=1, Female=0
df_clean['gender'] = df_clean['gender'].map({'M': 1, 'F': 0}).astype('Int64')

# Encode race
def categorize_race(race_str):
    race_str = str(race_str).upper()
    if 'WHITE' in race_str:
        return 1
    elif race_str in ['UNKNOWN', 'PATIENT DECLINED TO ANSWER', 'UNABLE TO OBTAIN']:
        return None
    else:
        return 0
    
df_clean['race'] = df_clean['race'].apply(categorize_race)
df_clean = df_clean.dropna(subset=['race'])
df_clean['race'] = df_clean['race'].astype('Int64')

# Reset index
df_clean.reset_index(drop=True, inplace=True)

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10501 entries, 0 to 10500
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   target           10501 non-null  Int64  
 1   gender           10501 non-null  Int64  
 2   age              10501 non-null  Int64  
 3   race             10501 non-null  Int64  
 4   heart_rate_min   10501 non-null  float64
 5   heart_rate_max   10501 non-null  float64
 6   heart_rate_mean  10501 non-null  float64
 7   glucose_min      10501 non-null  float64
 8   glucose_max      10501 non-null  float64
 9   glucose_mean     10501 non-null  float64
 10  sbp_min          10501 non-null  float64
 11  sbp_max          10501 non-null  float64
 12  sbp_mean         10501 non-null  float64
 13  cci              10501 non-null  Int64  
 14  apsiii           10501 non-null  Int64  
dtypes: Int64(6), float64(9)
memory usage: 1.3 MB


In [232]:
# All unique races to categorize into White vs Non-White

q("""
SELECT DISTINCT race
FROM `physionet-data.mimiciv_3_1_derived.icustay_detail`
""")

,race
0,UNKNOWN
1,WHITE
2,BLACK/AFRICAN AMERICAN
3,UNABLE TO OBTAIN
4,ASIAN - CHINESE
5,HISPANIC/LATINO - DOMINICAN
6,OTHER
7,HISPANIC OR LATINO
8,NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER
9,WHITE - OTHER EUROPEAN


In [233]:
df_clean.describe()

,target,gender,age,race,heart_rate_min,heart_rate_max,heart_rate_mean,glucose_min,glucose_max,glucose_mean,sbp_min,sbp_max,sbp_mean,cci,apsiii
count,10501.0,10501.0,10501.0,10501.0,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.0,10501.0
mean,0.286925,0.546424,67.594896,0.741929,76.143320,112.076374,91.766183,112.952290,667.071231,279.179604,83.978760,142.249357,110.697500,5.836016,58.903914
std,0.452348,0.497864,15.822325,0.437594,16.807689,22.532738,17.206940,43.538389,21812.812536,6730.117271,16.553104,23.328985,14.410510,3.085413,23.689218
min,0.0,0.0,18.0,0.0,3.000000,46.000000,40.538462,6.000000,18.000000,15.000000,0.108000,63.500000,51.875000,0.0,4.0
25%,0.0,0.0,58.0,0.0,64.000000,96.000000,79.230769,87.000000,123.000000,107.500000,75.000000,126.000000,101.080000,4.0,42.0
50%,0.0,1.0,69.0,1.0,75.000000,111.000000,91.076923,107.000000,158.000000,131.800000,84.000000,139.000000,108.269231,6.0,55.0
75%,1.0,1.0,80.0,1.0,87.000000,126.000000,103.560000,131.000000,220.000000,170.000000,93.000000,155.000000,117.888889,8.0,72.0
max,1.0,1.0,103.0,1.0,147.000000,277.000000,162.428571,950.000000,999999.000000,500037.500000,184.000000,336.000000,217.750000,19.0,180.0


##### Comments:
These are obvious errors, corroborated by the feature's std. 29812.04 and 7379.842060 respectively 
- glucose_max: 1.276100e+06 (MAX)	
- glucose_mean: 500048.500000 (MAX)

In [244]:
df_clean = df_clean[
    (df_clean['glucose_max'] < 2000) &
    (df_clean['glucose_mean'] < 2000) &
    (df_clean['glucose_min'] > 0) 
    # (df_clean['sbp_min'] > 20) &
    # (df_clean['heart_rate_min'] > 20) &
    # (df_clean['heart_rate_max'] < 250) &
    # (df_clean['apsiii'] <= 163) &
    # (df_clean['sbp_max'] <= 300)
]

print(f"Rows removed: {len(df_unique) - len(df_clean)}")
print(f"Rows remaining: {len(df_clean)}")

df_clean.describe()

Rows removed: 1464
Rows remaining: 10492


,target,gender,age,race,heart_rate_min,heart_rate_max,heart_rate_mean,glucose_min,glucose_max,glucose_mean,sbp_min,sbp_max,sbp_mean,cci,apsiii
count,10492.0,10492.0,10492.0,10492.0,10492.000000,10492.000000,10492.000000,10492.000000,10492.000000,10492.000000,10492.000000,10492.000000,10492.000000,10492.0,10492.0
mean,0.286695,0.546321,67.59064,0.741803,76.138296,112.064335,91.757822,112.977221,189.709112,146.233766,83.988463,142.247570,110.698956,5.835303,58.888105
std,0.452239,0.497873,15.823009,0.437664,16.808368,22.525783,17.207083,43.539356,110.977074,56.601350,16.548771,23.332648,14.413482,3.08559,23.670321
min,0.0,0.0,18.0,0.0,3.000000,46.000000,40.538462,6.000000,18.000000,15.000000,0.108000,63.500000,51.875000,0.0,4.0
25%,0.0,0.0,58.0,0.0,64.000000,96.000000,79.222222,87.000000,123.000000,107.500000,75.000000,126.000000,101.079231,4.0,42.0
50%,0.0,1.0,69.0,1.0,75.000000,111.000000,91.066667,107.000000,158.000000,131.750000,84.000000,139.000000,108.269231,6.0,55.0
75%,1.0,1.0,80.0,1.0,87.000000,126.000000,103.553783,131.000000,220.000000,170.000000,93.000000,155.000000,117.892473,8.0,72.0
max,1.0,1.0,103.0,1.0,147.000000,277.000000,162.428571,950.000000,1888.000000,950.000000,184.000000,336.000000,217.750000,19.0,180.0


### Cohort Distribution

In [247]:
df_sepsis = df_clean.copy()

In [248]:
features = {
    'Race': {
        'White':     lambda df: df['race'] == 1,
        'Non-White': lambda df: df['race'] == 0,
    },
    'Sex': {
        'Male':   lambda df: df['gender'] == 1,
        'Female': lambda df: df['gender'] == 0,
    }
}

def distribution_table(df, target_col, features):
    rows = []
    
    neg = df[df[target_col] == 0]
    pos = df[df[target_col] == 1]
    
    for feature, categories in features.items():
        rows.append({'Characteristic': feature, 'Overall': '', 'Neg. Class (Survived)': '', 'Pos. Class (Died)': ''})
        for label, condition in categories.items():
            overall_n = condition(df).sum()
            neg_n = condition(neg).sum()
            pos_n = condition(pos).sum()
            
            overall_pct = overall_n / len(df) * 100
            neg_pct = neg_n / len(neg) * 100
            pos_pct = pos_n / len(pos) * 100
            
            rows.append({
                'Characteristic': f'  {label}',
                'Overall': f'{overall_n:,} ({overall_pct:.1f}%)',
                'Neg. Class (Survived)': f'{neg_n:,} ({neg_pct:.1f}%)',
                'Pos. Class (Died)': f'{pos_n:,} ({pos_pct:.1f}%)'
            })
    
    table = pd.DataFrame(rows).set_index('Characteristic')
    
    table.columns = [
        f'Overall (N={len(df):,})',
        f'Neg. Class (N={len(neg):,})',
        f'Pos. Class (N={len(pos):,})'
    ]
    
    return table

distribution_table(df_sepsis, 'target', features)

,"Overall (N=10,492)","Neg. Class (N=7,484)","Pos. Class (N=3,008)"
Characteristic,,,
Race,,,
White,"7,783 (74.2%)","5,536 (74.0%)","2,247 (74.7%)"
Non-White,"2,709 (25.8%)","1,948 (26.0%)",761 (25.3%)
Sex,,,
Male,"5,732 (54.6%)","4,070 (54.4%)","1,662 (55.3%)"
Female,"4,760 (45.4%)","3,414 (45.6%)","1,346 (44.7%)"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df_sepsis.to_csv('/content/drive/MyDrive/df_sepsis.csv', index=False)